In [5]:
def davies_meyer(encrypt, l_key, l_message):
    def hash(key_message):
        key = key_message[:l_key]
        message = key_message[l_key:l_key + l_message]
        
        encrypted_message = encrypt(key, message)
        hashed_message = bytes(a ^ b for (a, b) in zip(encrypted_message, key))
        return hashed_message
    return hash

In [6]:
def identity(key, value):
    return value

comprension = davies_meyer(identity, 16, 16)
key = b'aca hay un mensa'
text = b'y aca hay otro m'

value = comprension(key + text)
value

b'\x18C\x00C\tA\x11A\x0cNO\x19\x17\x01S\x0c'

In [76]:
import math

def pad(message, l_block):
    message_length = len(message)
    n_blocks = math.ceil(message_length / l_block)
    missing_bytes = n_blocks * l_block - message_length
    
    added_bytes = b''
    if missing_bytes:
        added_bytes += b'1'
        added_bytes += b'0'*(missing_bytes - 1)
    
    coded_length = message_length % (2 ** l_block)
    last_block = coded_length.to_bytes(l_block,'big')
    padded_message = message + added_bytes + last_block

    return padded_message
    

In [78]:
pad(b'asdfasdfdsafsaasdfa', 16)

b'asdfasdfdsafsaasdfa1000000000000\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x13'

In [49]:
def merkle_damgard(IV, comp, l_block):
    def hash(message):
        padded_message = pad(message, l_block)
        h = IV
        for i in range(0, len(padded_message), l_block):
            block = padded_message[i: i + l_block]
            h = comp(h + block)
        return h
    return hash


In [50]:
comprension = davies_meyer(identity, 16, 16)
hash = merkle_damgard(b'123456789012345', comprension, 16)
h1 = hash(b'serasefasfasdfasdfasdfassdfasd')
h1

b'2!!#61%8;7#%1dh'